# 导入模块

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import random
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 连接数据库

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 查询数据

In [3]:
sql = '''
select 
om.create_time, om.order_number, tprm.order_id, tmu.true_name 客户姓名, tol.delivery_province_name, tol.delivery_city_name, cc.name as channel_name, ymos.refund_date, ymos.reality_refund_date
,tmu.id_card_num, om.status
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
-- 用户表
left join db_digua_business.t_member_user tmu on tmu.id=om.user_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 订单物流表
left join db_digua_business.t_order_logistics tol on tol.order_id=om.id
-- 资方订单分期表
left join (select order_id, refund_date, reality_refund_date from db_rent.ya_merchant_order_stages where sort=2) ymos on ymos.order_id=om.id
where date_format(om.create_time, '%Y-%m-%d')>='2025-10-01'
and date_format(om.create_time, '%Y-%m-%d')<='2025-10-31'
'''
df = query(sql)

In [4]:
t_date = datetime.now().date()
df.loc[:, 'now_day'] = pd.to_datetime(datetime.now().date())
df.loc[:, '下单日期'] = pd.to_datetime(df.create_time.dt.date)
df.loc[:, 'reality_refund_date'] = df.reality_refund_date.dt.date
df.loc[:, 'overdue_day'] = np.where(df.reality_refund_date.isna(), np.where(df.refund_date<df.now_day, df.now_day-df.refund_date, 0), 
                                    np.where(df.reality_refund_date>df.refund_date, df.reality_refund_date-df.refund_date, 0))
df.loc[:, 'overdue_day'] = df.overdue_day.apply(lambda x: x.days)
df_new = df[(df.overdue_day>0)&(df.reality_refund_date.isna())&(df.status==4)]
# 获取需要字段
df_new.loc[:, '用户来源'] = df_new.channel_name.fillna('未知来源')
df_new.loc[:, '省/市'] = df_new.delivery_province_name+df_new.delivery_city_name
df_new.loc[:, '年龄'] = df_new.apply(lambda x: clean.get_age(x['id_card_num'],x['下单日期']), axis=1)
df_new.loc[:, '性别'] = df_new.apply(lambda x: clean.get_gender(x.id_card_num), axis=1)


In [5]:
sql_name = '''
        SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
        FROM db_digua_business.t_user_verify_order_record tuvor
        left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
        where tuvor.del_flag = 0 ORDER BY tuvor.update_time
        '''
df_name = query(sql_name)


In [6]:
df_name = df_name.sort_values(by='update_time', ascending=False).groupby('order_id').head(1)
# 创建分配人映射关系 
name_dict = {
        '小张': '李巧玲',
        '小周': '李巧凤',
        '小南': '刘三妹',
        '小何': '何静',
        '小谢': '谢金凤',
        '小咪': '杨健',
        '小慧': '林思慧',
        '小滢': '胡彩滢',
        '小星': '廖丽敏',
        '小兰': '黄兰娟',
        '小晚': '周莹',
        '小芳': '罗芳'
        }
name_list = ['罗文龙', '何静', '刘三妹', '杨健', '林思慧', '胡彩滢', '周汉鸿', '廖丽敏','洪佳祥', '黄兰娟', '周莹', '邹巧巧','冯二洋']
# 匹配映射字典
df_name.loc[:, '信审员'] = df_name.分配人.apply(lambda x: name_dict[x] if str(x).startswith('小') and str(x) in name_dict.keys() else x)
# df_name = df_name[df_name.信审员.isin(name_list)]

In [7]:
# 关联分配人
df_all = df_new.merge(df_name[['order_id', '信审员']], on='order_id', how='left')
# 获取台账数据
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025")
df_all = df_all.merge(df_ck[['订单号', '型号']],left_on="order_number", right_on="订单号", how='left')
df_all.drop_duplicates(subset=["订单号"],inplace=True)
df_all.loc[:, '手机型号'] = df_all.型号.apply(lambda x: str(x).strip(r'无锁|有锁|带锁|95新|准新|99新'))
df_all_new = df_all[['信审员', 'order_number', '客户姓名', '手机型号', '省/市', '用户来源', '性别', '年龄']].rename(columns={'order_number': '订单编号'})
df_all_new = df_all_new[df_all_new.信审员.isin(name_list)]
df_all_new.to_excel(f'F:/需求/总需求/10月份首逾_{t_date}.xlsx', index=False)